## Explore and cluster the neighborhoods in Toronto assignment - Part I

Code to scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

For web scrapping, will be using the Pandas lib as shown below

In [73]:
import requests
import pandas as pd

Wikipedia page URL to be scrapped - "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [78]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Get the URL contents and store it in pandas dataframe

In [79]:
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

The dataframe consist of three columns: PostalCode, Borough, and Neighborhood

In [80]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
print(df.shape)
df.head()

(288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Will process only the cells that have an assigned Borough and ignore cells with a Borough that is Not assigned

In [92]:
#Check how many Borough are not assigned any value
df[df['Borough'] == 'Not assigned'].count()

PostalCode      77
Borough         77
Neighborhood    77
dtype: int64

In [85]:
df1 = df[df['Borough'] != 'Not assigned']
print(df1.shape)
df1.head()

(211, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [91]:
#Verify that all Borough which were not assigned are removed from data
df1[df1['Borough'] == 'Not assigned'].count()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

If a cell has a Borough but a Not assigned Neighborhood, then the neighborhood will be the same as the Borough

In [86]:
#check how many Neighborhood are not assigned
df1.loc[df1['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


In [87]:
df1.loc[df1['Neighborhood'] == 'Not assigned' , 'Neighborhood'] = df1['Borough']
print(df1.shape)
df1.head()

(211, 3)


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [93]:
#verify that Neighborhood that was not assigned gets value of Borough
df1.loc[df1['Neighborhood'] == df1['Borough']]

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Queen's Park


More than one Neighborhood can exist in one PostalCode area. Such rows will be combined into one row with the Neighborhoods separated with a comma

In [98]:
df1[df1['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [99]:
foo = lambda a: ", ".join(a) 
df2 = df1.groupby(['PostalCode', 'Borough']).agg({'Neighborhood': foo}).reset_index()
print(df2.shape)
df2.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [100]:
#verify that different Neighborhoods with same PostalCode are combined in one row
df2[df2['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


So here is the generated data frame after all the processing, now lets check the shape of that

In [102]:
print("Num of rows in generated data frame = ", df2.shape[0])

Num of rows in generated data frame =  103
